# Project Set Up

In [1]:
# Setup
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import sample_data

In [2]:
!git clone https://github.com/btgraham/SparseConvNet.git

Cloning into 'SparseConvNet'...
remote: Enumerating objects: 1904, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 1904 (delta 8), reused 11 (delta 4), pack-reused 1880
Receiving objects: 100% (1904/1904), 907.06 KiB | 2.93 MiB/s, done.
Resolving deltas: 100% (1357/1357), done.


In [3]:
%cd SparseConvNet/
!bash develop.sh

/content/SparseConvNet
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running develop
running egg_info
creating sparseconvnet.egg-info
writing sparseconvnet.egg-info/PKG-INFO
writing dependency_links to sparseconvnet.egg-info/dependency_links.txt
writing top-level names to sparseconvnet.egg-info/top_level.txt
writing manifest file 'sparseconvnet.egg-info/SOURCES.txt'
package init file 'sparseconvnet/SCN/__init__.py' not found (or not a regular file)
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:369: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
writing manifest file 'sparseconvnet.egg-info/SOURCES.txt'
running build_ext
building 'sparseconvnet.SCN' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/sparseconvnet
creating build/temp.linux-x86_64-3.7/sp

# Varying max-pooling layers $\ell$
## (Table 1 in the paper)

In [4]:
!bash examples/Assamese_handwriting/process.sh

Streaminguitvoer ingekort tot de laatste 5000 regels.
Extracting  48.25.txt                                                     41%  OK 
Extracting  49.25.txt                                                     41%  OK 
Extracting  5.25.txt                                                      41%  OK 
Extracting  50.25.txt                                                     41%  OK 
Extracting  51.25.txt                                                     41%  OK 
Extracting  52.25.txt                                                     41%  OK 
Extracting  53.25.txt                                                     41%  OK 
Extracting  54.25.txt                                                     41%  OK 
Extracting  55.25.txt                                                     41%  OK 
Extracting  56.25.txt                                                     41%  OK 
Extracting  57.25.txt     

In [5]:
import os
import pickle
train = []
test = []
p = [45,9,3,34,31,25,23,17,22,37,27,2,10,13,7,42,4,8,33,11,12,1,39,38,
     36,20,14,21,40,24,32,5,35,18,44,41,30,28,29,19,15,26,6,16,43]


def rescaleCharacter(c):
    cc = torch.cat(c, 0)
    m = cc.min(0)[0]
    s = (cc.max(0)[0] - m).float()
    for i in range(len(c)):
        c[i] = (torch.div((c[i] - m.expand_as(c[i])).float(), 
            s.expand_as(c[i])) * 255.99).byte()
    return c

for char in range(1, 183 + 1):
    for writer in range(0, 36):
        exec('c=' + open('tmp/'+str(char)+'.'+str(p[writer])+'.py', 'r').read())
        train.append({'input': rescaleCharacter(c), 'target': char})
        
for char in range(1, 183 + 1):
    for writer in range(36, 45):
        exec('c=' + open('tmp/'+str(char)+'.'+str(p[writer])+'.py', 'r').read())
        test.append({'input': rescaleCharacter(c), 'target': char})

os.mkdir('pickle')
pickle.dump(train, open('pickle/train.pickle', 'wb'))
pickle.dump(test, open('pickle/test.pickle', 'wb'))

In [6]:
import sparseconvnet as scn
%cd examples/Assamese_handwriting

/content/SparseConvNet/examples/Assamese_handwriting


In SparseConvNet/examples/Assamese_handwriting/data.py on line 57 adjust jitter=8 to jitter=0.

For $\ell = 3$ until $\ell = 7$, run the code below. Note that the current architecture is for $\ell = 7$, the other architecture can be found in the appendix on our blog.

In [ ]:
from data import get_iterators

class Model(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.sparseModel = scn.Sequential( 
          scn.SparseVggNet(2, 3, [
            ['C', 30], ['C', 30], 'MP',
            ['C', 60], ['C', 60], 'MP',
            ['C', 90], ['C', 90], 'MP',
            ['C', 120], ['C', 120], 'MP',
            ['C', 150], ['C', 150], 'MP',
            ['C', 180], ['C', 180], 'MP',
            ['C', 210], ['C', 210], 'MP']),
          scn.Convolution(2, 210, 240, 5, 1, False),
          scn.BatchNormReLU(240),
          scn.SparseToDense(2, 240))
        self.spatial_size= self.sparseModel.input_spatial_size(torch.LongTensor([1, 1]))
        self.inputLayer = scn.InputLayer(2,self.spatial_size,2)
        self.linear = nn.Linear(240, 183) # 183 Indo-Aryan characters in the Online Handwritten Assamese Characters Dataset

    def forward(self, x):
        x = self.inputLayer(x)
        x = self.sparseModel(x)
        x = x.view(-1, 240)
        x = self.linear(x)
        return x

model = Model()
scale=63
dataset = get_iterators(model.spatial_size, scale)
print('l=7')
print('Input spatial size:', model.spatial_size, 'Data scale:', scale)

scn.ClassificationTrainValidate(
    model, dataset,
    {'n_epochs': 30,
     'initial_lr': 0.1,
     'lr_decay': 0.05,
     'weight_decay': 1e-4,
     'use_cuda': torch.cuda.is_available(),
     'check_point': False, })

6588 1647
Replicating dataset: 1 epoch = 10 iterations of the dataset; 10 x 6588 = 65880 training samples


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


l=7
Input spatial size: tensor([767, 767]) Data scale: 63
{'n_epochs': 30, 'initial_lr': 0.1, 'lr_decay': 0.05, 'weight_decay': 0.0001, 'use_cuda': False, 'check_point': False, 'momentum': 0.9, 'test_reps': 1, 'epoch': 1}
#parameters 3349953


# Varying feature set size $M$
## (Table 2 in the paper)

Run the code below for jitter = 0 as well as jitter = 8. Adjusting this can be done in SparseConvNet/examples/Assamese_handwriting/data.py on line 57.

In [ ]:
from data import get_iterators

class Model(nn.Module):
    def _init_(self):
        nn.Module._init_(self)
        self.sparseModel = scn.Sequential(
          scn.SparseVggNet(2, 3, [
            ['C', 20], ['C', 20], 'MP',
            ['C', 40], ['C', 40], 'MP',
            ['C', 60], ['C', 60], 'MP',
            ['C', 80], ['C', 80], 'MP']),
          scn.Convolution(2, 80, 100, 5, 1, False),
          scn.BatchNormReLU(100),
          scn.SparseToDense(2, 100))
        self.spatial_size= self.sparseModel.input_spatial_size(torch.LongTensor([1, 1]))
        self.inputLayer = scn.InputLayer(2,self.spatial_size,2)
        self.linear = nn.Linear(100, 183)

    def forward(self, x):
        x = self.inputLayer(x)
        x = self.sparseModel(x)
        x = x.view(-1, 100)
        x = self.linear(x)
        return x

model = Model()
scale=63
dataset = get_iterators(model.spatial_size, scale)
print('Input spatial size:', model.spatial_size, 'Data scale:', scale)

scn.ClassificationTrainValidate(
    model, dataset,
    {'n_epochs': 30,
     'initial_lr': 0.1,
     'lr_decay': 0.05,
     'weight_decay': 1e-4,
     'use_cuda': torch.cuda.is_available(),
     'check_point': False, })